# Analysis

In [1]:
import time
import tqdm
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re
import lemmy # For lemmatization
import nltk
from nltk.stem import SnowballStemmer
import itertools

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords = stopwords.words('danish')
stemmer = SnowballStemmer("danish")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load datasets

In [3]:
ft_sygeplej2x = pd.read_csv('ft_sygeplej2x.csv')
dr_sygeplej2x = pd.read_csv('dr_sygeplej2x.csv')
tv2_sygeplej2x = pd.read_csv('tv2_sygeplej2x.csv')

ft = ft_sygeplej2x.copy() 
dr = dr_sygeplej2x.copy() 
tv2 = tv2_sygeplej2x.copy()

### Cleaning
- lower case (already done)
- remove non-alphanumeric characters
- remove numbers

In [4]:
def cleaner(document):
    document = document.lower() #To lower case
    document = re.sub(r'[^\w\s]','', document) #Remove non-alphanumeric characters
    document = re.sub(r'[^\D+]','', document) #Remove numeric characters
    return document

### Pre-processing
- Tokenize
- Remove stopwords
- Stemming

In [6]:
def pre_processing(df): 
    tokens = [nltk.tokenize.word_tokenize(df['content'][i]) for i in range(0, len(df))]
    tokens = list(itertools.chain(*tokens)) # list of lists to list
    
    nostop = [i for i in tokens if i not in stopwords]

    stemmed = [stemmer.stem(word) for word in nostop]

# Bag of words

In [7]:
def BoW(df): 
    count = CountVectorizer()
    df_array = df['content']
    bag = count.fit_transform(df_array)
    
    count_array = bag.toarray() #Make the bag to an array
    matrix = pd.DataFrame(data=count_array,columns = count.get_feature_names())
    
    return matrix


# 2-gram

In [8]:
def two_gram(df):
    count = CountVectorizer(ngram_range=(2,2)) #Choose only 2-grams

    df_array = df['content']
    bag = count.fit_transform(df_array)

    count_array = bag.toarray() #Make the bag to an array
    matrix = pd.DataFrame(data=count_array,columns = count.get_feature_names()) #Input the bag and the words into a dataframe
    
    return matrix


# tf-idf

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

def tfidf(df):
    ############################## bag #################################
    count = CountVectorizer(ngram_range=(2,2)) #Choose only 2-grams
    
    df_array = df['content']
    bag = count.fit_transform(df_array)
    ############################## bag #################################
    
    tfidf = TfidfTransformer()
    bag_tfidf = tfidf.fit_transform(bag) 

    tfidf_array = bag_tfidf.toarray() #Make the bag to an array
    matrix_tfidf = pd.DataFrame(data=tfidf_array,columns = count.get_feature_names())
    return matrix_tfidf

# Apply functions

In [ ]:
sample1 = tv2[0:3].copy()

In [25]:
df = tv2

In [26]:
df['content'] = df['content'].apply(cleaner)

In [27]:
pre_processing(df)

In [28]:
frame = BoW(df)
frame.to_csv('tv2_bow.csv')


C:\Users\kim\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


KeyboardInterrupt: 

In [20]:
frame = two_gram(df)
frame.to_cs

C:\Users\kim\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,__von_schwartz__ vi,_valentina_burtseva_ vi,_årige kvinde,_årige mand,aa og,aaaaah jeg,aaaaalt for,aabel og,aabenhus samatvdk,aabenraa derfor,...,सथ सन,सन जवन,सब चपट,समय गय,हcoronajihad nizamuddinterrorists,हम जग,ชวตตดถำ ชวตรอดแลว,ชวตรอดแลว พาหมปากลบบาน,ถำหลวง ชวตตดถำ,พาหมปากลบบาน thailandcaverescue
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3604,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3605,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
tfidf(df)

C:\Users\kim\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,__von_schwartz__ vi,_valentina_burtseva_ vi,_årige kvinde,_årige mand,aa og,aaaaah jeg,aaaaalt for,aabel og,aabenhus samatvdk,aabenraa derfor,...,सथ सन,सन जवन,सब चपट,समय गय,हcoronajihad nizamuddinterrorists,हम जग,ชวตตดถำ ชวตรอดแลว,ชวตรอดแลว พาหมปากลบบาน,ถำหลวง ชวตตดถำ,พาหมปากลบบาน thailandcaverescue
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
